<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_answer_KCBERT_nsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질문에 대한 답변을 선별하는 코드(BERT Next Sentence Prediction)(로컬)

In [ ]:
!pip install soynlp
!pip install datasets
import random
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from soynlp.word import WordExtractor
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Mounted at /content/drive


#모델 학습

MLM으로 비지도 학습

In [ ]:
#훈련 데이터 불러오기
pretrain_dataset_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.xlsx'
df_pretrain = pd.read_excel(pretrain_dataset_path)

In [ ]:
#Huggingface의 원본 모델 로드
#BertForMaskedLM 구조에 맞게 모델 불러오기
HUGGINGFACE_MODEL_PATH = 'beomi/kcbert-base'

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = BertForMaskedLM.from_pretrained(HUGGINGFACE_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
#GPU 사용 코드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
model.to(device)

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#soynlp tokenizer를 Pretrain 데이터로 학습
pretrain_list = df_pretrain['text'].to_list()

word_extractor = WordExtractor()
word_extractor.train(pretrain_list)
word_score_table = word_extractor.extract()

In [ ]:
print(word_score_table)

In [ ]:
# 예제로 몇 가지 단어 점수 확인
for word, score in word_scores.items():
    print(f"{word}: {score.cohesion_forward:.2f}")

In [ ]:
# 높은 cohesion 점수를 가진 단어들을 선택하여 단어사전 구성
threshold = 0.1  # 점수 기준 설정
new_vocab = [word for word, score in word_scores.items() if score.cohesion_forward > threshold]

# 선택된 단어 출력
print(new_vocab)

In [ ]:
# 새로운 단어 토크나이저에 추가
num_added_toks = tokenizer.add_tokens(new_vocab)
print(f"Added {num_added_toks} tokens")

# 모델의 임베딩 크기 조정
model.resize_token_embeddings(len(tokenizer))

In [ ]:
#MLM 데이터셋 준비
corpus = pretrain_list
tokenized_corpus = [tokenizer.encode_plus(text, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_tensors='pt') for text in corpus]

In [ ]:
#MLM을 위한 데이터셋의 일부를 [MASK] 토큰으로 교체하는 함수
def mask_input_ids(input_ids, tokenizer, mlm_probability=0.15):
    labels = input_ids.clone()
    # 모든 토큰을 마스킹 확률에 따라 랜덤하게 선택
    probability_matrix = torch.full(labels.shape, mlm_probability)
    special_tokens_mask = [tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # PyTorch에서 사용하는 masked loss 계산을 위해 마스킹 되지 않은 부분은 -100으로 설정

    # `[MASK]` 토큰으로 대체
    input_ids[masked_indices] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
    return input_ids, labels

In [ ]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# 트레이너 생성
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                'labels': torch.stack([f[1] for f in data])},
    train_dataset=tokenized_corpus  # 이 부분은 실제 데이터셋으로 대체 필요
)

# 훈련 시작
trainer.train()

In [ ]:
#google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Pretrained_Model'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

Labeled NSP 데이터셋으로 파인튜닝

In [ ]:
#훈련 데이터 불러오기
finetune_dataset_path = '/content/drive/My Drive/KcBERT_finetune_dataset.xlsx'
df_finetune = pd.read_excel(finetune_dataset_path)

In [ ]:
#MLM으로 훈련된 모델 로드
#BertForNextSentencePrediction 구조에 맞게 모델 불러오기
pretrained_model_load_path = '/content/drive/My Drive/Pretrained_Model'

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_load_path)
pretrained_model = BertForNextSentencePrediction.from_pretrained(pretrained_model_load_path)

In [ ]:
class NSPDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        sentence_pair = {
            'sentence1': row['sent1'],
            'sentence2': row['sent2']
        }
        inputs = self.tokenizer(sentence_pair['sentence1'], sentence_pair['sentence2'],
                                max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        inputs = {key: val.squeeze() for key, val in inputs.items()}
        inputs['labels'] = torch.LongTensor([int(row['label'])])
        return inputs

In [ ]:
#NSP 데이터셋 준비
train_dataset = NSPDataset(df_finetune, pretrained_tokenizer, max_length = pretrained_model.config['max_position_embeddings'])

In [ ]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 디렉터리
    num_train_epochs=3,              # 에폭 수
    per_device_train_batch_size=8,   # 배치 크기
    warmup_steps=500,                # 워밍업 스텝 수
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로그 디렉터리
    logging_steps=10,
)

# 트레이너 초기화
trainer = Trainer(
    model=pretrained_model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()

In [ ]:
#google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Finetuned_Model'

pretrained_model.save_pretrained(save_path)
pretrained_tokenizer.save_pretrained(save_path)

#모델 사용

In [ ]:
#NSP로 훈련된 모델 로드
finetuned_model_load_path = '/content/drive/My Drive/Finetuned_Model'

finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_load_path)
finetuned_model = BertForNextSentencePrediction.from_pretrained(finetuned_model_load_path)

In [ ]:
#카카오톡 데이터 불러오기
file_path = '/content/drive/My Drive/judge_question_result_short.xlsx'

In [ ]:
#카카오톡 대화내용을 데이터프레임으로 받기
df = pd.read_excel(file_path)

#질문 딕셔너리, 답변 목록 리스트(이중 리스트) 생성
questions, answer_lists = {}, []

#질문으로 판별된 텍스트를 새 데이터프레임으로 생성
df_question = df[df['label'] == 'question']

print(df_question)
#새 데이터프레임의 index, text를 question 딕셔너리에 저장
questions = {text : index for (index, text) in zip(df_question.index, df_question['text'])}
print(questions)

In [ ]:
# 추론할 문장 쌍
sentence1 = "오늘 점심은 매우 맛있었다."
sentence2 = "내일은 무엇을 먹을까?"
# inputs = tokenizer(sentence1, sentence2, return_tensors = 'pt')
# print(inputs)

sentence_dict = {"오늘 점심은 매우 맛있었다." : "내일은 무엇을 먹을까?", "나는 너를 사랑해" : "너는 나를 사랑하지만, 나는 너를 사랑하지 않아", "안녕하세요? 전학생입니다" : "이번 설연휴는 날씨가 정말 춥겠습니다."}

# # 배치 처리 후 토크나이징
# inputs = [tokenizer(sentence1, sentence2, return_tensors='pt') for sentence1, sentence2 in sentence_dict.items()]

#튜플로 묶은 뒤 배치처리
batched_data = [(sentence1, sentence2) for sentence1, sentence2 in sentence_dict.items()]
inputs = tokenizer(batched_data, padding=True, truncation=True, return_tensors = 'pt')
print(inputs)

#모델을 평가 모드로 설정
model.eval()

# 추론 실행
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)
    print(outputs)
    print(logits)
    print(probs)

print(probs[1])
print(probs[1][1])
# # 결과 해석
# is_continuation = probs[0][1] > 0.5
# if is_continuation:
#     print("두 문장은 연속적입니다.")
# else:
#     print("두 문장은 연속적이지 않습니다.")

{'input_ids': tensor([[    2,  8564, 18485,  4057, 11212,  1306,  4188,  8217,    17,     3,
         12122,  4057, 12449, 10865,  4109,    32,     3,     0,     0,     0],
        [    2,  8616, 14669,  9004,  4032,     3,  9039, 15919,  9004,  9082,
            15,  8616, 14669,  9004,  7998, 11969,     3,     0,     0,     0],
        [    2, 19017,  8482,    32,  2525,  9068,  8074,     3,  8175,  1849,
          4132,  4984,  4008, 19972,  4009,  8050,  2939,  9560,    17,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
NextSentencePredictorOutput(loss=None, logits=t

In [ ]:
# Ensure using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

In [ ]:
#중복 질문 제거를 동시에 수행하기 위한 새로운 알고리즘

#실험을 위한 하이퍼파라미터 설정
#후보군으로 삼을 텍스트 개수 범위
text_range = 20
#답변 목록에 추가할지 기준이 되는 레이블값
standard = 0.6

#질문-답변 쌍 딕셔너리 생성
qa_pair_dictionary = {index : {'질문' : question, '답변' : []} for index, question in zip(df_question.index, df_question['text'])}

#질문 속 질문인지 판별할 때 사용할 불리언
in_question_texts = False

all_texts = {index : item for index, item in zip(df.index, df['text'])}

for index, item in tqdm(all_texts.items(), desc = 'Processing Answer to Question'):

  candidate_qa_list = [] #현재 텍스트의 소속을 판정할 (질문-답변 딕셔너리) 리스트
  candidate_qa_index_list = []
  in_question_texts = True if df.iloc[index]['label'] == 'question' else False #판별할 텍스트가 질문인지 검사

  start = 0 if index < 20 else index - text_range - 1 #인덱스가 20 미만일 경우 검사 범위 조정
  for i in range(start, index): #현재 텍스트가 소속될 질문의 범위
    candidate = qa_pair_dictionary.get(i, None) #qa_pair_dictionary에서 i 인덱스에 해당하는 질답 딕셔너리 가져오기
    if candidate != None:
      candidate_qa_list.append(candidate) #결과 리스트에서 최대 확률인 질문을 인덱싱하기 위해 인덱스를 포함한 딕셔너리를 append
      candidate_qa_index_list.append(i)

  if len(candidate_qa_list) == 0:
    continue

  #데이터를 튜플로 묶은 뒤 배치처리
  batched_data = [(qa_dict['질문'] + ' '.join(qa_dict['답변']), item) for qa_dict in candidate_qa_list]
  inputs = tokenizer(batched_data, padding = True, truncation = 'only_first', return_tensors = 'pt')
  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

  outputs = [{'scores' : value, 'index' : index} for value, index in zip(probabilities.tolist(), candidate_qa_index_list)]

  # 'scores'의 첫 번째 값(expected label일 확률)에 따라 내림차순으로 정렬
  sorted_output = sorted(outputs, key = lambda x : x['scores'][1], reverse=True)

  output = sorted_output[0] #텍스트가 소속될 질문

  if output['scores'][1] > standard:
    #모순이 발생하지 않기 위해서 질문 속 질문으로 판별된 경우 즉시 qa_pair_dictionary에서 해당 질문을 삭제해야 함.
    if in_question_texts == True:
        del qa_pair_dictionary[index]

    qa_pair_dictionary[output['index']]['답변'].append(item)


  # print('\n',qa_pair_dictionary)
  # print('\n',used_question_index)


In [ ]:
# #데이터프레임 가공
# df_question.rename(columns = {'text' : 'question'})
# df_question['answer'] = answer_lists

data = {question : answer for question, answer in zip(questions.keys(), answer_lists)}
print(data)
data_result= [{'question' : key, 'answer' : value} for key, value in data.items()]
df_result = pd.DataFrame(data = data_result, columns = ['question', 'answer'])
print(df_result)

In [ ]:
#.xlsx 파일로 google drive에 저장
save_path = '/content/drive/My Drive/judge_answer_result.xlsx'

df_result.to_excel(save_path)